In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import catboost as ctb
from sklearn.metrics import log_loss
import numpy as np
import category_encoders as ce

In [75]:
train_set = pd.read_csv('train_prepared.csv', encoding='utf-8')
test_set = pd.read_csv('test_prepared.csv', encoding='utf-8')

In [76]:
X = train_set.drop(columns=['Target', 'Opportunity_ID'])
y = train_set.Target

In [77]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

clf = xgb.XGBClassifier()

clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [78]:
len(X_train.columns)

53

In [79]:
f_imp = clf.feature_importances_
f = X_train.columns
fts = ['Opportunity_ID', 'Target']

for i in range (len(f_imp)):
    print("{: >10}\t\t{: >50}".format(f[i],f_imp[i]))
    if f_imp[i]>=0.0095:
        fts.append(f[i])

Pricing, Delivery_Terms_Quote_Appr		                              0.006891307421028614
Pricing, Delivery_Terms_Approved		                              0.007873987779021263
Bureaucratic_Code_0_Approval		                              0.008105112239718437
Bureaucratic_Code_0_Approved		                              0.006480127573013306
Quote_Type		                              0.010113364085555077
Convertibility		                              0.013810486532747746
Total_Amount_Sum_USD		                              0.011271057650446892
Total_Taxable_Amount_USD		                              0.012472603470087051
Year_Creation		                              0.012832541950047016
Month_Creation		                              0.009849204681813717
Year_Delivery		                              0.012726370245218277
Month_Delivery		                              0.007966133765876293
Days_Passed		                               0.01573769561946392
Wait_Time_Days		                               0.0652289

In [80]:
len(fts)-2

35

In [81]:
train_filtered = train_set.loc[:, fts]

X = train_filtered.drop(columns=['Opportunity_ID', 'Target'])
y = train_filtered.Target

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [82]:
X_train

,Quote_Type,Convertibility,Total_Amount_Sum_USD,Total_Taxable_Amount_USD,Year_Creation,Month_Creation,Year_Delivery,Days_Passed,Wait_Time_Days,Delivery_Window,...,Account_Type_target,Bureaucratic_Code_target,Region_target,Account_Name_target,Opportunity_Owner_target,Or,Last_Modified_By_Owners,Expiry_Days_Create,Expiry_Days_Modified,Days_Modified_Diff_to_End_Delivery
4886,1,1.131098,9.935561e+04,9.935561e+04,2017,6,2017,245.0,5.0,0.0,...,0.681871,0.593111,0.462820,0.561798,0.482405,0,False,1.0,-244.0,240.0
9098,1,1.131079,8.016525e+04,8.016525e+04,2018,11,2019,2.0,66.0,0.0,...,0.681871,0.283333,0.462820,0.847059,0.434185,0,True,-67.0,-69.0,-64.0
1109,1,0.008990,2.699831e+03,2.699831e+03,2016,2,2016,298.0,9.0,NaN,...,0.681871,0.593111,0.676810,0.843590,0.621086,0,False,2.0,-296.0,NaN
4766,1,1.000000,2.040000e+04,0.000000e+00,2017,6,2017,251.0,35.0,0.0,...,0.484398,0.593111,0.444896,0.333840,0.285467,0,False,28.0,-223.0,216.0
4686,1,1.131086,1.977421e+06,1.977421e+06,2017,5,2018,260.0,312.0,60.0,...,0.484398,0.593111,0.462820,0.121472,0.573316,0,False,37.0,-223.0,-112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9785,1,1.131094,3.506958e+05,7.013916e+05,2015,11,2016,414.0,43.0,5.0,...,0.681871,0.593111,0.462820,0.481481,0.482405,0,False,13.0,-401.0,366.0
7763,1,1.131079,7.749307e+04,7.749307e+04,2018,5,2018,168.0,161.0,66.0,...,0.484398,0.593111,0.462820,0.000001,0.272040,0,True,NaN,NaN,-59.0
5218,1,0.008990,3.843396e+04,3.843396e+04,2017,6,2017,223.0,40.0,0.0,...,0.296500,0.593111,0.676810,0.999996,0.767594,0,False,NaN,NaN,183.0
1346,1,1.000000,3.263400e+05,3.263400e+05,2016,3,2016,255.0,179.0,11.0,...,0.681871,0.593111,0.676810,0.852941,1.000000,0,False,30.0,-225.0,65.0


In [83]:
#def objective(space):

    #warnings.filterwarnings(action='ignore', category=DeprecationWarning)
#    classifier = xgb.XGBClassifier(n_estimators = space['n_estimators'],
#                            max_depth = int(space['max_depth']),
#                            learning_rate = space['learning_rate'],
#                            gamma = space['gamma'],
#                            min_child_weight = space['min_child_weight'],
#                            subsample = space['subsample'],
#                            colsample_bytree = space['colsample_bytree']
#                            )
    
#    classifier.fit(X_train, y_train)

    # Applying k-Fold Cross Validation
#    from sklearn.model_selection import cross_val_score
#    accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
#    CrossValMean = accuracies.mean()

#    loss = log_loss(y_test, classifier.predict_proba(X_test))
    
#    print("-----------------")
#    print("log_loss:", loss)
#    print("CV mean:", CrossValMean)
#    print("-----------------")

#    return{'loss':loss, 'status': STATUS_OK }

#space = {
#    'max_depth' : hp.choice('max_depth', range(5, 30, 1)),
#    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
#    'n_estimators' : hp.choice('n_estimators', range(20, 250, 5)),
#    'gamma' : hp.quniform('gamma', 0, 0.50, 0.01),
#    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
#    'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
#    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.01)}

#trials = Trials()
#best = fmin(fn=objective,
#            space=space,
#            algo=tpe.suggest,
#            max_evals=250,
#            trials=trials)

#print("Best: ", best)

In [84]:
best = {'colsample_bytree': 0.84, 
        'gamma': 0.49, 'learning_rate': 0.04, 'max_depth': 23,
        'min_child_weight': 6.0, 'n_estimators': 55, 'subsample': 0.84}

In [85]:
final = xgb.XGBClassifier(**best)

In [86]:
final.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.84, gamma=0.49, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.04, max_delta_step=0, max_depth=23,
              min_child_weight=6.0, missing=nan, monotone_constraints='()',
              n_estimators=55, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.84,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [87]:
preds_x = final.predict_proba(X_test)

In [88]:
from sklearn.metrics import f1_score

In [89]:
print(log_loss(y_test, preds_x))
f1_score(y_test, final.predict(X_test))

0.3142163922065191


0.8794664125774178

In [90]:
test_filtered = test_set.loc[:, fts]

In [91]:
test_preds = final.predict_proba(test_filtered.drop(columns=['Opportunity_ID', 'Target']))

In [92]:
print(log_loss(test_filtered.Target, test_preds))
f1_score(test_filtered.Target, final.predict(test_filtered.drop(columns=['Target', 'Opportunity_ID'])))

0.4554985124420496


0.8087500000000001

In [93]:
preds = pd.DataFrame(test_preds[:, 1])
preds['Opportunity_ID'] = test_filtered.Opportunity_ID.to_list()
preds['Real'] = test_filtered.Target.to_list()

In [94]:
sub = preds.loc[:, ['Opportunity_ID', 0]]
sub.columns = ['Opportunity_ID', 'Target']
sub

,Opportunity_ID,Target
0,10689,0.692545
1,10690,0.461894
2,10691,0.284488
3,10692,0.509998
4,10693,0.905786
...,...,...
1562,12364,0.924402
1563,12365,0.221347
1564,12366,0.190413
1565,12367,0.255683


In [95]:
sub.to_csv('predictions/xgb_hyperopt.csv', index=False)